In [ ]:
import glob
import os 

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import files
import occupancy
import wait_times
import vehicles

# General Utils

In [ ]:
SCENARIOS_PATH = r"E:/Files/CodeProjects/MATSim/Scenarios/"
RUN_DIR = "runner-tests/OneToFive80x800.5/OneToFive80x800"
PATH = os.path.join(SCENARIOS_PATH, RUN_DIR)


def fmt_timedelta(x):
    ts = x.total_seconds()
    hours, remainder = divmod(ts, 3600)
    minutes, _ = divmod(remainder, 60)
    return ('{:02d}:{:02d}').format(int(hours), int(minutes)) 


# Wait time utils

In [ ]:
r = files.iterate_algorithms(PATH, wait_times.get_wait_stats)
pd.DataFrame(r).stack().unstack(level=-2)

# Population Utils

In [ ]:
def get_origin_destination_data(alg_path):
    pop_filename = "Population*.xml.DistanceInfo.csv"
    path = glob.glob(os.path.join(alg_path, pop_filename))[0]
    df = pd.read_csv(path, names=["person", "distance"], skiprows=1)
    return pd.cut(df["distance"], include_lowest=True, bins=10)


out = get_origin_destination_data(PATH+"/DRT")
ax = out.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(12,8))
ticks = [(round(c.left), round(c.right)) for c in out.cat.categories]
ax.set_xticklabels(ticks)
ax.set_xlabel("Home to Work Distance [m]")
ax.set_ylabel("Number of People")
plt.xticks(rotation=45)
plt.show()

# Fleet distance

In [ ]:
r = files.iterate_algorithms(PATH, vehicles.get_distances_stats)
pd.DataFrame(r).stack().unstack(level=-2) / 1000

# Ocuupancy utils

In [ ]:
df = files.iterate_algorithms(PATH, occupancy.get_ocucpancy_aggregation)
df = pd.DataFrame(df).stack().unstack(level=-2).fillna(0)
df.style.format({col: '{:,.2f}%'
                 for col in df.columns})

In [ ]:
axes = files.iterate_algorithms(PATH, occupancy.get_occupancy_graphs)
fig = plt.figure()
map(fig.add_axes, axes)
fig